In [8]:
import os
from time import sleep
import json
import argparse
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
import base64

In [2]:
parser = argparse.ArgumentParser(description="test LLM planning abilities")
parser.add_argument(
        "--dim",
        type=int,
        default=1,
        help="dimension of the problem 1 or 2 (for 1D or 2D)",
)
parser.add_argument(
        "--prompt_type",
        type=str,
        default='text',
        help="type of prompt",
    )
parser.add_argument(
        "--color",
        type=int,
        default=0,
        help="Use color images (1) or not (0)",
    )
parser.add_argument(
        "--model_name",
        type=str,
        default='all',
        help="num",
    )
parser.add_argument(
        "--key",
        type=str,
        default='',
        help="num",
    )

args_string = '--dim=2 --prompt_type=image --color=1'
args_list = args_string.split(' ')
args = parser.parse_args(args_list)

In [16]:
if args.model_name=='all':
    model_name = [
        'gpt-3.5-turbo',
        'gpt-4o-mini',
        'gpt-4o-2024-08-06'
    ]
else:
    model_name = args.model_name
# model_name='gpt-3.5-turbo'
# model_name='gpt-4o-mini'
model_name='gpt-4o-2024-08-06'
if not args.key:
    apikey_filepath = '../.openai_key.txt'
    with open(apikey_filepath, 'r') as f:
        key = f.read()
else:
    key = args.key

openai_client = ChatOpenAI(api_key=key, model_name=model_name)

In [4]:
# Load Data
if args.dim == 1:
    examples_data_dir = '../data/brick_1D_50'
elif args.dim == 2:
    examples_data_dir = '../data/brick_2D_50'

with open(os.path.join(examples_data_dir, 'data.json')) as f:
    data = json.load(f)

zeroshot_prompt = 'Lets think step by step, and provide the answer in the format of a sequence of bricks by a comma in the last sentence.'


In [5]:
if args.prompt_type.lower() in ['text', 'txt']:
    res_list = []

    for i, item in enumerate(data):
        print('step: ', i+1)
        response = openai_client(messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": zeroshot_prompt + '\n\n' + 'Question:' + '\n' + item['data'] + '\nAnswer:\n'}
        ],
        max_tokens=2048,
        temperature=0)
        res = response.content

        dict_res = {'pred': res, 'label': item['label']}
        res_list.append(dict_res)


In [17]:
if args.prompt_type.lower() in ['image','img']:

    def encode_image(image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    

    image_subdir = 'images'
    if args.color:
        color='color'
    else:
        color='bw'

    img_prompt = 'The image shows a set of bricks that can be placed on top of each other. Now we have to get a specific brick. The bricks must now be grabbed from top to bottom, and if the lower brick is to be grabbed, the upper brick must be removed first. How to get brick {t}?'
    res_list = []
    i = 0

    for i, item in enumerate(data):
        print('step: ', i+1)
        image_path = os.path.join(examples_data_dir, image_subdir, f'img_{i}_{color}.png')
        image_base64 = encode_image(image_path)

        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", zeroshot_prompt + img_prompt.format(t=item['target'])),
                (
                    "user",
                    [
                        {
                            "type": "image_url",
                            "image_url": {"url": "data:image/jpeg;base64,{image_data}"},
                        }
                    ],
                ),
            ]
        )
        chain = prompt | openai_client
        res = chain.invoke({'image_data':image_base64}).content
        dict_res = {'pred': res, 'label': item['label']}
        res_list.append(dict_res)

step:  1
step:  2
step:  3
step:  4
step:  5
step:  6
step:  7
step:  8
step:  9
step:  10
step:  11
step:  12
step:  13
step:  14
step:  15
step:  16
step:  17
step:  18
step:  19
step:  20
step:  21
step:  22
step:  23
step:  24
step:  25
step:  26
step:  27
step:  28
step:  29
step:  30
step:  31
step:  32
step:  33
step:  34
step:  35
step:  36
step:  37
step:  38
step:  39
step:  40
step:  41
step:  42
step:  43
step:  44
step:  45
step:  46
step:  47
step:  48
step:  49
step:  50


In [18]:
if args.prompt_type.lower() in ['img','image']:
    colorname = '_color' if args.color else '_bw'
else:
    colorname = ''

output_path = os.path.join('../data/results/', f'{args.dim}D_{args.prompt_type}{colorname}_{model_name}.json')
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(output_path, 'w') as outfile:
    json.dump(res_list, outfile)

print(f'Results saved to {output_path}')

Results saved to ../data/results/2D_image_color_gpt-4o-2024-08-06.json
